CONEXIÓN MINIO

In [1]:
from minio import Minio
from minio.error import S3Error

In [2]:
def conectar_minio(endpoint='localhost:9000', 
                  access_key=None, 
                  secret_key=None, 
                  secure=False):
    """
    Establece una conexión con un servidor MinIO.
    
    Parámetros:
    -----------
    endpoint : str, opcional
        Dirección del servidor MinIO (por defecto 'localhost:9000')
    access_key : str, requerido
        Usuario/access key para la autenticación
    secret_key : str, requerido
        Contraseña/secret key para la autenticación
    secure : bool, opcional
        Si es True, usa HTTPS. Si es False, usa HTTP (por defecto False)
    
    Retorna:
    --------
    Minio
        Cliente MinIO conectado
    
    Excepciones:
    ------------
    ValueError
        Si no se proporcionan access_key o secret_key
    S3Error
        Si hay un error al conectar con el servidor MinIO
    """
    
    if not access_key or not secret_key:
        raise ValueError("Se requieren access_key y secret_key para la conexión")
    
    try:
        # Crear cliente MinIO
        cliente = Minio(
            endpoint,
            access_key=access_key,
            secret_key=secret_key,
            secure=secure
        )
        
        # Verificar conexión listando los buckets (opcional)
        buckets = cliente.list_buckets()
        print(f"Conexión exitosa a MinIO en {endpoint}")
        print(f"Buckets disponibles: {[bucket.name for bucket in buckets]}")
        
        return cliente
    
    except S3Error as err:
        print(f"Error al conectar con MinIO: {err}")
        raise

Subir archivos Minio

In [4]:
from io import BytesIO, StringIO
import pandas as pd
from minio.error import S3Error  # Asegúrate de importar S3Error

def guardar_df_en_minio(minio_client, df, bucket_name, ruta_destino, 
                        formato='parquet', crear_bucket=False):
    """
    Guarda un DataFrame directamente en un bucket de MinIO.
    """
    
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not isinstance(df, pd.DataFrame):
        raise ValueError("El parámetro df debe ser un pandas.DataFrame")
    if not bucket_name or not ruta_destino:
        raise ValueError("bucket_name y ruta_destino son requeridos")
    
    formatos_soportados = {
        'parquet': {
            'mime': 'application/parquet',
            'writer': lambda buffer: df.to_parquet(buffer, index=False),
            'extension': '.parquet',
            'buffer_type': BytesIO
        },
        'csv': {
            'mime': 'text/csv',
            'writer': lambda buffer: df.to_csv(buffer, index=False),
            'extension': '.csv',
            'buffer_type': StringIO
        },
        'json': {
            'mime': 'application/json',
            'writer': lambda buffer: df.to_json(buffer, orient='records'),
            'extension': '.json',
            'buffer_type': BytesIO
        },
        'excel': {
            'mime': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            'writer': lambda buffer: df.to_excel(buffer, index=False),
            'extension': '.xlsx',
            'buffer_type': BytesIO
        }
    }
    
    formato = formato.lower()
    if formato not in formatos_soportados:
        raise ValueError(f"Formato '{formato}' no soportado. Use: {list(formatos_soportados.keys())}")
    
    if not ruta_destino.lower().endswith(formatos_soportados[formato]['extension']):
        ruta_destino += formatos_soportados[formato]['extension']
    
    try:
        if crear_bucket and not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"Bucket '{bucket_name}' creado exitosamente")

        if not minio_client.bucket_exists(bucket_name):
            raise S3Error(f"El bucket '{bucket_name}' no existe", bucket_name, None, 404)
        
        # Crear buffer en memoria
        buffer = formatos_soportados[formato]['buffer_type']()
        formatos_soportados[formato]['writer'](buffer)

        # Ajuste para CSV (convertir a BytesIO)
        if formato == 'csv':
            buffer.seek(0)
            data = BytesIO(buffer.getvalue().encode('utf-8'))
            length = len(data.getvalue())
        else:
            buffer.seek(0)
            data = buffer
            length = buffer.getbuffer().nbytes
        
        # Subir a MinIO
        minio_client.put_object(
            bucket_name=bucket_name,
            object_name=ruta_destino,
            data=data,
            length=length,
            content_type=formatos_soportados[formato]['mime']
        )
        
        ruta_completa = f"{bucket_name}/{ruta_destino}"
        print(f"DataFrame guardado exitosamente en: {ruta_completa}")
        return ruta_completa

    except S3Error as err:
        print(f"Error al guardar DataFrame en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el DataFrame: {e}")
        raise


Leer archivo desde minio Minio

In [6]:
import PyPDF2
import docx

def extraer_archivo_minio(minio_client, bucket_name, ruta_archivo, tipo_archivo):
    """
    Extrae un archivo de MinIO y lo retorna en el formato adecuado.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde está el archivo
    ruta_archivo : str
        Ruta completa del archivo dentro del bucket
    tipo_archivo : str
        Tipo de archivo a extraer ('pdf', 'word', 'excel', 'csv', 'parquet')
    
    Retorna:
    --------
    Depende del tipo de archivo:
    - 'pdf': Texto extraído (str)
    - 'word': Documento de python-docx
    - 'excel': DataFrame de pandas
    - 'csv': DataFrame de pandas
    - 'parquet': DataFrame de pandas
    
    Excepciones:
    ------------
    ValueError
        Si los parámetros son inválidos o el tipo no es soportado
    S3Error
        Si hay un error al acceder al archivo o el bucket no existe
    """
    
    # Validaciones iniciales
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not bucket_name or not ruta_archivo or not tipo_archivo:
        raise ValueError("bucket_name, ruta_archivo y tipo_archivo son requeridos")
    
    tipo_archivo = tipo_archivo.lower()
    tipos_soportados = ['pdf', 'word', 'excel', 'csv', 'parquet']
    
    if tipo_archivo not in tipos_soportados:
        raise ValueError(f"Tipo de archivo '{tipo_archivo}' no soportado. Use: {tipos_soportados}")
    
    try:
        # Obtener el objeto de MinIO
        response = minio_client.get_object(bucket_name, ruta_archivo)
        data = BytesIO(response.read())
        data.seek(0)
        
        # Procesar según el tipo de archivo
        if tipo_archivo == 'pdf':
            # Extraer texto de PDF
            pdf_reader = PyPDF2.PdfReader(data)
            text = "\n".join([page.extract_text() for page in pdf_reader.pages])
            return text
        
        elif tipo_archivo == 'word':
            # Retornar documento de Word
            return docx.Document(data)
        
        elif tipo_archivo in ['excel', 'csv', 'parquet']:
            # Leer con pandas según el formato
            if tipo_archivo == 'excel':
                return pd.read_excel(data)
            elif tipo_archivo == 'csv':
                return pd.read_csv(data)
            elif tipo_archivo == 'parquet':
                return pd.read_parquet(data)
        
    except S3Error as err:
        print(f"Error al acceder al archivo en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el archivo {ruta_archivo}: {e}")
        raise
    finally:
        response.close()
        response.release_conn()

Generar ruta 

In [10]:
from datetime import datetime

def generar_ruta_fecha(separador='/', fecha=None):
    """
    Genera un string con la fecha en formato año/mes/día.
    
    Parámetros:
    -----------
    separador : str, opcional
        Carácter separador entre componentes (por defecto '/')
    fecha : datetime, opcional
        Fecha específica a formatear (si None, usa fecha actual)
    
    Retorna:
    --------
    str
        String con el formato 'YYYY{separador}MM{separador}DD'
    """
    # Usar fecha actual si no se proporciona una específica
    fecha_a_usar = fecha if fecha is not None else datetime.now()
    
    # Formatear la fecha
    ruta_fecha = fecha_a_usar.strftime(f"%Y{separador}%m{separador}%d")
    
    return ruta_fecha

In [11]:
ruta = generar_ruta_fecha()
print(ruta)  

2025/05/05


In [12]:
import re
def clean_special_characters(text):
    """
    Limpia caracteres especiales y no deseados en un texto o lista de textos.

    Args:
        text (str | list): Texto o lista de textos a limpiar.

    Returns:
        str: Texto limpio.
    """
    if isinstance(text, list):
        # Si es una lista, convertirla en una sola cadena separada por espacios
        text = ' '.join(text)
    
    if isinstance(text, str):
        # Reemplazar \uf0d8 con viñeta estándar
        text = text.replace('\uf0d8', '•')
        # Eliminar corchetes y otros caracteres especiales
        text = re.sub(r'[\[\]\"]', '', text)
        # Eliminar caracteres no ASCII imprimibles
        text = re.sub(r'[^\x20-\x7E]', '', text)
        return text.strip()  # Eliminar espacios en blanco adicionales

    return text  # Si no es cadena ni lista, devolver sin cambios

In [13]:
def remove_points(text):
    """
    Remueve los puntos de un texto.

    Parámetros:
    - text (str o list): Texto a limpiar.

    Retorno:
    - str: Texto limpio.
    """
    if not isinstance(text, (str, list)):
        raise ValueError("El texto debe ser una cadena o una lista")

    if isinstance(text, list):
        text = ' '.join(text)

    return text.replace('.', '')

In [20]:
import dateparser
def transformate_date(date):
    """
    Transforma una fecha en el formato 'Enero 10 de 2024' a '30/12/2022'.

    Parámetros:
    - fecha (str): Fecha a transformar.

    Retorno:
    - str: Fecha transformada.
    """
    fecha_parseada = dateparser.parse(date)
    if fecha_parseada:
        return fecha_parseada.strftime('%d/%m/%Y')
    else:
        return None

In [22]:
def transformar_tipo_identificacion(df):
    """
    Transforma la columna 'TipoIdentificacion' según la edad y crea una nueva columna 'id_paciente'.

    Parámetros:
    - df (DataFrame): DataFrame que contiene las columnas 'edad', 'Identificacion' y 'TipoIdentificacion'.

    Retorno:
    - df (DataFrame): DataFrame modificado con la nueva columna 'id_paciente'.
    """
    df['Edad'] = df['Edad'].astype(int)
    # Asignar 'CC' o 'TI' según la edad
    df.loc[df['Edad'] >= 18, 'TipoIdentificacion'] = 'CC'
    df.loc[df['Edad'] < 18, 'TipoIdentificacion'] = 'TI'
    # Renombrar la columna 'Identificacion'
    df.rename(columns={'Identificacion': 'NumeroIdentificacion'}, inplace=True)
    df['TipoIdentificacion'] = df['TipoIdentificacion'].astype(str)  # Asegurarse de que sea una cadena
    df['NumeroIdentificacion'] =df['NumeroIdentificacion'].astype(str)  # Asegurarse de que sea una cadena
    # Crear la nueva columna 'id_paciente' concatenando 'TipoIdentificacion' y 'NumeroIdentificacion'
    df['id_paciente'] = df['TipoIdentificacion'] + df['NumeroIdentificacion']

    return df

In [16]:
def get_table_columns(table_name, connection_params):
    """
    Obtiene las columnas de una tabla en PostgreSQL.

    Args:
        table_name (str): Nombre de la tabla en PostgreSQL.
        connection_params (dict): Parámetros de conexión a PostgreSQL.

    Returns:
        list: Lista con los nombres de las columnas de la tabla.
    """
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()

        # Consultar los nombres de las columnas de la tabla
        query = f"""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_name = '{table_name}';
        """
        cursor.execute(query)
        columns = [row[0] for row in cursor.fetchall()]

        return columns

    except Exception as e:
        print(f"Error al obtener las columnas de la tabla {table_name}: {e}")
        return []
    finally:
        if conn:
            cursor.close()
            conn.close()

In [17]:
def insert_into_info_lab(df, connection_params):
    """
    Inserta los datos de un DataFrame en la tabla 'infolab' de PostgreSQL,
    considerando solo las columnas que coincidan entre los nombres del DataFrame y los de la tabla.

    Args:
        df (pd.DataFrame): DataFrame con los datos a insertar.
        connection_params (dict): Diccionario con los parámetros de conexión a PostgreSQL.
    """
    # Obtener las columnas de la tabla 'infolab'
    table_columns = get_table_columns('infolab', connection_params)
    
    if not table_columns:
        raise ValueError("No se pudieron obtener las columnas de la tabla 'infolab'.")

    # Seleccionar las columnas que coincidan entre el DataFrame y la tabla
    matching_columns = [col for col in df.columns if col in table_columns]
    
    if not matching_columns:
        raise ValueError("No hay columnas coincidentes entre el DataFrame y la tabla 'infolab'.")

    # Conexión a PostgreSQL
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()
        
        # Crear la consulta INSERT dinámica
        columns_str = ", ".join(matching_columns)
        placeholders_str = ", ".join(["%s"] * len(matching_columns))
        insert_query = f"INSERT INTO infolab ({columns_str}) VALUES ({placeholders_str})"

        # Preparar los datos para la inserción
        values = df[matching_columns].values.tolist()

        # Ejecutar la consulta
        cursor.executemany(insert_query, values)
        conn.commit()

        print(f"Se insertaron {cursor.rowcount} filas en la tabla 'infolab'.")
    
    except Exception as e:
        print(f"Error al insertar los datos: {e}")
        conn.rollback()
    finally:
        if conn:
            cursor.close()
            conn.close()

In [ ]:
def transformate_df(df):
    """Transforma un DataFrame limpiando caracteres especiales."""
    try:
        # Definir las funciones y columnas a aplicar
        df[['Identificacion', 'Numero de Informe', 'Fecha de Informe', 'Fecha de Toma de Muestra', 'Nombre del Paciente']] = df[
            ['Identificacion', 'Numero de Informe', 'Fecha de Informe', 'Fecha de Toma de Muestra', 'Nombre del Paciente']].astype(str)
        funciones = {
            'clean_special_characters': ['Diagnostico'],
            'remove_points': ['Identificacion', 'Numero de Informe', 'Fecha de Informe', 'Fecha de Toma de Muestra', 'Nombre del Paciente'],
            'transformate_date': ['Fecha de Informe', 'Fecha de Toma de Muestra']
        }
        
        # Aplicar las funciones a las columnas
        for funcion, columnas in funciones.items():
            for columna in columnas:
                df[columna] = df[columna].apply(eval(funcion))
        
        # Transformar el tipo de identificación
        df = transformar_tipo_identificacion(df)
        # Renombrar columnas del DataFrame para que coincidan con la tabla en la base de datos
        df.rename(columns={
            'Fecha de Toma de Muestra': 'fecha_toma_muestra',
            'Fecha de Ingreso': 'fecha_ingreso',
            'Fecha de Informe': 'fecha_informe',
            'Entidad': 'entidad',
            'EPS': 'eps',
            'Servicio': 'servicio',
            'Muestra Remitida': 'muestra_remitida',
            'Descripcion Macroscopica': 'descripcion_macroscopica',
            'Descripcion Microscopica': 'descripcion_microscopica',
            'Diagnostico': 'diagnostico',
            'Comentario': 'comentario',
            'llavePaciente': 'id_paciente',
            'Archivo': 'archivo',
            'Historia':'Identificacion'
        }, inplace=True)
        
        # Agregar la columna 'fuente' con un valor por defecto (si es necesario)
        df['fuente'] = 'Fernando_sanson'  # Cambia 'nombre_fuente' por el valor que corresponda
        return df
    except KeyError as e:
        print(f"Error: La columna {e} no se encuentra en el DataFrame.")
    except Exception as e:
        print(f"Error al momento de transdormar el df: {e}")

In [ ]:
def clean_entidad_field(entidad):
    """
    Limpia el campo `Entidad` eliminando cualquier información relacionada con
    "FECHA DE INGRESO" y "FECHA DE INFORME".

    Args:
        entidad (str): Texto del campo `Entidad` a limpiar.

    Returns:
        str: Texto limpio del campo `Entidad`.
    """
    if not isinstance(entidad, str):
        return entidad  # Si no es una cadena, devolver tal cual
    
    # Expresión regular para encontrar y eliminar las fechas
    cleaned_entidad = re.sub(r'FECHA DE INGRESO:.*?(FECHA DE INFORME:.*)?$', '', entidad, flags=re.IGNORECASE).strip()
    return cleaned_entidad


In [ ]:
def process_identificacion_column(df):
    """
    Divide la columna 'Identificacion' en 'TipoIdentificacion' y 'NumeroIdentificacion',
    renombra 'Identificacion' a 'LlavePaciente', y elimina los espacios en 'LlavePaciente'.

    Args:
        df (pd.DataFrame): DataFrame con la columna 'Identificacion'.

    Returns:
        pd.DataFrame: DataFrame modificado con las nuevas columnas.
    """
    # Separar los dos primeros caracteres como 'TipoIdentificacion'
    df['TipoIdentificacion'] = df['Identificacion'].str[:2].str.strip()
    
    # Separar el resto del texto como 'NumeroIdentificacion'
    df['NumeroIdentificacion'] = df['Identificacion'].str[3:].str.strip()
    
    # Renombrar la columna 'Identificacion' a 'LlavePaciente'
    df.rename(columns={'Identificacion': 'id_paciente'}, inplace=True)
    
    # Eliminar espacios en 'LlavePaciente' y convertir a minúsculas
    df['id_paciente'] = df['id_paciente'].str.replace(r'\s+', '', regex=True).str.upper()

    
    return df

In [ ]:
def transformar_df_lab_Jimenez( df):
    """Trasforma el DataFrame """
    try:
        # limpia la columna "Entidad"
        df['Entidad'] = df['Entidad'].apply(clean_entidad_field)
        # limpia la columna diagnotico
        df['Diagnostico'] = df['Diagnostico'].apply(clean_special_characters)
        df = process_identificacion_column(df)
        # Renombrar columnas del DataFrame para que coincidan con la tabla en la base de datos
        df.rename(columns={
            'Fecha de Toma de Muestra': 'fecha_toma_muestra',
            'Fecha de Ingreso': 'fecha_ingreso',
            'Fecha de Informe': 'fecha_informe',
            'Entidad': 'entidad',
            'EPS': 'eps',
            'Servicio': 'servicio',
            'Muestra Remitida': 'muestra_remitida',
            'Descripcion Macroscopica': 'descripcion_macroscopica',
            'Descripcion Microscopica': 'descripcion_microscopica',
            'Diagnostico': 'diagnostico',
            'Comentario': 'comentario',
            'llavePaciente': 'id_paciente',
            'Archivo': 'archivo'
        }, inplace=True)
        
        # Agregar la columna 'fuente' con un valor por defecto (si es necesario)
        df['fuente'] = '091'  # Cambia 'nombre_fuente' por el valor que corresponda
        return df
    except KeyError as e:
        print(f"Error: La columna {e} no se encuentra en el DataFrame.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Error al transformar el df: {e}")

In [ ]:
def transformar_df_lab_citopat(df):
    try:
        # Definir las funciones y columnas a aplicar
        df[['identificacion',  'Fecha de Informe', 'Fecha de Toma de Muestra', 'Nombre del Paciente']] = df[
            ['Identificacion',  'Fecha de Informe', 'Fecha de Toma de Muestra', 'Nombre del Paciente']].astype(str)
        funciones = {
            'clean_special_characters': ['Diagnostico'],
            'remove_points': ['Identificacion',  'Fecha de Informe', 'Fecha de Toma de Muestra', 'Nombre del Paciente'],
            'transformate_date': ['Fecha de Informe', 'Fecha de Toma de Muestra']
        }
        
        # Aplicar las funciones a las columnas
        for funcion, columnas in funciones.items():
            for columna in columnas:
                df[columna] = df[columna].apply(eval(funcion))
    except Exception as e:
        print(f"Error al transformar el df: {e}")

In [ ]:
from psycopg2 import sql
from psycopg2.extras import execute_batch
import psycopg2

def cargar_dfpaciente(df, nombre_tabla, connection_params, columna_id='id_paciente', etiqueta_nuevo=1):
    """
    Carga un DataFrame a una tabla PostgreSQL, evitando duplicados y marcando registros nuevos con etiqueta=1.
    """
    conn = None
    registros_insertados = 0
    
    try:
        # 1. Conectar a la base de datos
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()
        
        # 2. Obtener columnas existentes en la tabla destino
        query_columnas = sql.SQL("""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = %s
        """)
        cursor.execute(query_columnas, [nombre_tabla])
        columnas_bd = [col[0] for col in cursor.fetchall()]
        
        # 3. Filtrar columnas del DataFrame que existan en la tabla
        columnas_comunes = [col for col in df.columns if col in columnas_bd]
        df_filtrado = df[columnas_comunes].copy()
        
        # 4. Agregar la columna 'etiqueta' con valor fijo etiqueta_nuevo (=1)
        df_filtrado['etiqueta'] = etiqueta_nuevo

        # 5. Preparar columnas para insertar
        columnas_insert = list(df_filtrado.columns)
        
        placeholders = ', '.join(['%s'] * len(columnas_insert))
        conflict_target = columna_id  # La columna que tiene restricción UNIQUE o PRIMARY KEY
        
        query_insert = sql.SQL("""
            INSERT INTO {tbl} ({cols}) 
            VALUES ({vals})
            ON CONFLICT ({conflict_col}) DO NOTHING
        """).format(
            tbl=sql.Identifier(nombre_tabla),
            cols=sql.SQL(', ').join(map(sql.Identifier, columnas_insert)),
            vals=sql.SQL(placeholders),
            conflict_col=sql.Identifier(conflict_target)
        )
        
        # 6. Insertar en bloques
        datos_a_insertar = [tuple(row) for row in df_filtrado[columnas_insert].values]
        execute_batch(cursor, query_insert, datos_a_insertar, page_size=100)
        
        registros_insertados = cursor.rowcount
        conn.commit()
        print(f"Insertados {registros_insertados} registros nuevos en {nombre_tabla}.")
        
        return (True, registros_insertados)
        
    except Exception as e:
        if conn:
            conn.rollback()
        print(f"Error al cargar datos en {nombre_tabla}: {str(e)}")
        return (False, 0)
        
    finally:
        if conn:
            conn.close()


In [ ]:
import pandas as pd

def agregar_fuente(df: pd.DataFrame, nombre_fuente: str) -> pd.DataFrame:
    """
    Agrega una columna 'fuente' al DataFrame con el valor proporcionado.

    Parámetros:
    - df: DataFrame original
    - nombre_fuente: nombre de la fuente a agregar en la nueva columna

    Retorna:
    - Un nuevo DataFrame con la columna 'fuente' agregada
    """
    df = df.copy()  # Evitar modificar el original
    df["fuente"] = nombre_fuente
    return df


In [ ]:


def agregar_observacion(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea una nueva columna 'observacion' a partir de la concatenación de:
    'comentarios', 'descripcion_microscopica', 'descripcion_macroscopica' y 'diagnostico',
    separando cada parte con el nombre de la columna en mayúsculas seguido de ':'.

    Si alguna de las columnas no existe, se agrega con valores vacíos.

    Parámetros:
    - df: DataFrame original

    Retorna:
    - DataFrame con la nueva columna 'observacion'
    """
    df = df.copy()

    # Lista de columnas necesarias
    columnas_necesarias = [
        "comentarios", 
        "descripcion_microscopica", 
        "descripcion_macroscopica", 
        "diagnostico"
    ]

    # Verificar y crear columnas faltantes con valores vacíos
    for col in columnas_necesarias:
        if col not in df.columns:
            df[col] = ""

    # Rellenar nulos con cadena vacía
    for col in columnas_necesarias:
        df[col] = df[col].fillna("")

    # Concatenar con formato
    df["observacion"] = (
        "COMENTARIOS: " + df["comentarios"].astype(str).str.strip() + "| " +
        "DESCRIPCION_MICROSCOPICA: " + df["descripcion_microscopica"].astype(str).str.strip() + "| " +
        "DESCRIPCION_MACROSCOPICA: " + df["descripcion_macroscopica"].astype(str).str.strip() + "| " +
        "DIAGNOSTICO: " + df["diagnostico"].astype(str).str.strip()
    ).str.strip()

    return df
